## Imports

In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from loguru import logger

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

c:\Users\timmy_3aupohg\anaconda3\envs\hidro_app\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Embeddings

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\timmy_3aupohg\AppData\Local\Temp\ipykernel_22704\3401734470.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


## VecDB Setup

In [3]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"
index = pc.Index(index_name)

In [4]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [5]:
def search_db(user_query: str) -> list:
    sim_docs =[]
    result = vector_store.similarity_search_with_score(
    user_query, k=3
    )
    for doc in result:
        sim_docs.append(doc[0].page_content)
    
    return sim_docs    

In [6]:
search = search_db(user_query="What is candidiasis?")

In [7]:
search

['Candidiasis\nThis patient’s tongue is infected with candidiasis.(Photo-\ngraph by Edward H. Gill, Custom Medical Stock Photo. Repro-\nduced by permission.)\nGEM -0625 to 1002 - C  10/22/03 6:10 PM  Page 646',
 'teristic forms of yeasts at various stages in the lifecycle.\nFungal blood cultures should be taken for patients\nsuspected of having deep organ candidiasis. Tissue biop-\nsy may be needed for a definitive diagnosis.\nTreatment\nV aginal candidiasis\nIn most cases, vaginal candidiasis can be treated\nsuccessfully with a variety of over-the-counter antifungal\ncreams or suppositories. These include Monistat, Gyne-\nLotrimin, and Mycelex. However, infections often recur.',
 'of which increase a patient’s susceptibility to infection.\nDescription\nV aginal candidiasis\nOver one million women in the United States devel-\nop vaginal yeast infections each year. It is not life-threat-\nening, but it can be uncomfortable and frustrating.\nOral candidiasis\nThis disorder, also known as

## LLM Setup

In [8]:
# ---------------------------- LLM --------------------------------------
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)

In [20]:
def medicalbot_ai(user_query: str, doc_list: list) -> str:
    template = """
    You are a medical consultant AI chatbot. Your role is to provide accurate and reliable answers to user questions based on the provided documents. Use the information from the `doc_list` to address the `user_query` thoroughly and correctly. Ensure that your response is:

    - **Accurate:** Base your answers solely on the information in the provided documents.
    - **Conversational:** Maintain a friendly and approachable tone.
    - **Mature and Consultancy-Oriented:** Present information in a professional and trustworthy manner.

    **Inputs:**
    1. `user_query`: {user_query} The question posed by the user.
    2. `doc_list`: {doc_list} A list of documents containing relevant information related to the user's question.

    **Instructions:**
    - Analyze the `user_query` and identify the key information needed to answer it.
    - Review the `doc_list` to find relevant information that addresses the query.
    - Construct a response that is clear, concise, and directly answers the user's question using the information from the documents.
    - Avoid introducing information not present in the `doc_list`.
    - If the `user_query` have nothing similar to what is in the `doc_list`, return document not found or something in an apologetic way, tell the user to ask for something related to the context.
    - If the  `user_query` is an empty string, respond with "Please provide a valid query.".
    - Maintain a tone that is both professional and empathetic, suitable for a consultancy setting.
    
    Return the answer as the only output. 
    Always make sure that you're returning the answer without any explanation. 
    The output should be the answer alone.
    Always return this: "Please provide a valid query." for empty query.
    """
    question_prompt = PromptTemplate(input_variables=["user_query", "doc_list"], template=template)
    initiator_router = question_prompt | llm | StrOutputParser()
    output = initiator_router.invoke({"user_query":user_query, "doc_list":doc_list})
    return output

In [10]:
answer_1 = medicalbot_ai(user_query="What is Candidiasis?", doc_list=search)

In [11]:
print(answer_1)

Candidiasis is an infection caused by yeast, with various forms, including vaginal candidiasis, which affects over one million women in the United States each year, and oral candidiasis, also known as thrush, which causes white, curd-like patches in the mouth or throat. It is not life-threatening but can be uncomfortable and frustrating. Treatment for vaginal candidiasis often involves over-the-counter antifungal creams or suppositories, such as Monistat, Gynelotrimin, and Mycelex, while deep organ candidiasis may require fungal blood cultures and tissue biopsy for a definitive diagnosis.


## Bringing all together

In [22]:
def medical_chatbot(user_query: str)-> str:
    logger.info("Searching for similar Docs in the Vec DB")
    doc_list = search_db(user_query=user_query)
    logger.info("Calling the medicalbot ai to get answer")
    answer = medicalbot_ai(user_query=user_query, doc_list=doc_list)
    logger.info("Returning the Final Answer")
    return answer

In [23]:
question_1 = "What is Candidiasis?"
question_2 = "what is Carpal tunnel syndrome?"
question_3 = "what is the treatment of Carpal tunnel syndrome?"
question_4 = "What is the treatment of Candidiasis?"

In [24]:
answer_1 = medical_chatbot(user_query=question_1)
print(answer_1)

2025-01-04 13:26:38.710 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-01-04 13:26:41.576 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:26:42.930 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Candidiasis is an infection caused by yeast, and it can manifest in different forms, such as vaginal candidiasis, which is a common condition affecting over one million women in the United States each year, and oral candidiasis, also known as thrush, which causes white, curd-like patches in the mouth or throat. It is not life-threatening but can be uncomfortable and frustrating. Treatment for vaginal candidiasis often involves over-the-counter antifungal creams or suppositories, while deep organ candidiasis may require more invasive diagnostic procedures like fungal blood cultures and tissue biopsies.


In [15]:
answer_2 = medical_chatbot(user_query=question_2)
print(answer_2)

2025-01-04 13:18:17.161 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-01-04 13:18:17.409 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:18:18.213 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Carpal tunnel syndrome is a disorder caused by compression at the wrist of the median nerve supplying the hand, causing numbness and tingling. The carpal tunnel is an area in the wrist where the bones and ligaments create a small passageway for the median nerve, which is responsible for both sensation and movement in the hand, particularly the thumb and first three fingers. When the median nerve is compressed, an individual's hand will feel as if it has "fallen asleep" or is numb and tingly.


In [16]:
answer_3 = medical_chatbot(user_query=question_3)
print(answer_3)

2025-01-04 13:18:18.230 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-01-04 13:18:18.600 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:18:19.635 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


The treatment of Carpal tunnel syndrome may involve controlling the symptoms with splinting and anti-inflammatory agents. In more advanced cases, injection of steroids into the wrist to decrease inflammation may be necessary. For the most severe cases, surgery may be required to decrease the compression of the median nerve and restore its normal function, which involves cutting the ligament that crosses the wrist, allowing the median nerve more room and decreasing compression. This surgery is often performed on an outpatient basis, with about 95% of people experiencing complete cessation of symptoms after the procedure.


In [17]:
answer_4 = medical_chatbot(user_query=question_4)
print(answer_4)

2025-01-04 13:18:19.672 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-01-04 13:18:20.174 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:18:21.511 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


The treatment of Candidiasis involves removing any catheters present in the patient and starting antifungal chemotherapy to prevent the spread of the disease. The choice of antifungal drug should be based on the patient's specific history and defense status. For vaginal candidiasis, over-the-counter antifungal creams or suppositories such as Monistat, Gyne-Lotrimin, and Mycelex can be used. In some cases, prescription antifungal drugs like terconazole may be necessary. For recurrent infections, women may need to take antifungal drugs on a preventive basis. Additionally, home remedies like vinegar douches or insertion of a paste made from Lactobacillus acidophilus powder into the vagina may be used. In cases of deep organ candidiasis, fungal blood cultures and tissue biopsies may be needed for a definitive diagnosis, and treatment should be tailored accordingly.


In [18]:
test_5 = medical_chatbot("What is Deep Learning?")
print(test_5)

2025-01-04 13:18:21.531 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-01-04 13:18:21.909 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:18:22.723 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


I'm afraid I couldn't find any information related to "Deep Learning" in the provided documents. It seems that the documents are focused on medical terminology and concepts, and do not cover topics related to artificial intelligence or machine learning. If you have any questions related to the medical field, I'd be happy to try and assist you. Please feel free to ask a question that's more relevant to the context of the documents, and I'll do my best to provide a helpful response.


In [21]:
test_5 = medical_chatbot("")
print(test_5)

2025-01-04 13:19:18.152 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB


2025-01-04 13:19:18.440 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-01-04 13:19:19.242 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Please provide a valid query.
